In [1]:
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from pymongo import MongoClient
import pymongo
import math
import random
import pandas as pd
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Add, Lambda, Concatenate, Dense
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import Sequence
from spektral.utils import normalized_adjacency
from spektral.layers import GCNConv
import unicodedata

#from scipy.sparse import dok_matrix


seed_value = 42
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

# Data load/generator

In [2]:
with open('data/dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)
    
client = MongoClient()
music = client['music_recommender']

In [3]:
def load_social(file_users, file_edges, users_ids):
    df_users = pd.read_csv(file_users, sep='\t', names=['id', 'user'])
    df_edges = pd.read_csv(file_edges, sep=' ', names=['origin', 'destination'])
    old_new = {}
    for _, r in tqdm(df_users.iterrows(), total=len(df_users)):
        if r['user'] in users_ids:
            old_new[r['id']] = users_ids[r['user']]
    social_graph = nx.DiGraph()
    social_graph.add_nodes_from(old_new.values())
    for _, r in tqdm(df_edges.iterrows(), total=len(df_edges)):
        if r['origin'] in old_new and r['destination'] in old_new:
            social_graph.add_edge(old_new[r['origin']], old_new[r['destination']])
    return social_graph

social_graph = load_social('lastfm_sn/lastfm.nodes', 'lastfm_sn/lastfm.edges', dataset['users'])

  0%|          | 0/136420 [00:00<?, ?it/s]

  0%|          | 0/1685524 [00:00<?, ?it/s]

In [4]:
'''
pos = nx.spring_layout(social_graph, seed=42)

nodes = nx.draw_networkx_nodes(social_graph, pos, node_color="indigo")
edges = nx.draw_networkx_edges(
    social_graph,
    pos,
    arrowstyle="->",
    arrowsize=10,
    width=2,
)

pc = mpl.collections.PatchCollection(edges)#, cmap=cmap)
#pc.set_array(edge_colors)
#plt.colorbar(pc)

ax = plt.gca()
ax.set_axis_off()
plt.show()
'''

'\npos = nx.spring_layout(social_graph, seed=42)\n\nnodes = nx.draw_networkx_nodes(social_graph, pos, node_color="indigo")\nedges = nx.draw_networkx_edges(\n    social_graph,\n    pos,\n    arrowstyle="->",\n    arrowsize=10,\n    width=2,\n)\n\npc = mpl.collections.PatchCollection(edges)#, cmap=cmap)\n#pc.set_array(edge_colors)\n#plt.colorbar(pc)\n\nax = plt.gca()\nax.set_axis_off()\nplt.show()\n'

In [5]:
nodes = set(social_graph.nodes)
ds = nx.to_undirected(social_graph)
to_visit = [nodes.pop()]
communities = []
com = []
while len(to_visit) > 0:
    c = to_visit.pop()
    com.append(c)
    next_v = {x for x in ds[c] if x in nodes}
    nodes = nodes - next_v
    to_visit.extend(next_v)
    if len(to_visit) == 0:
        communities.append(com)
        com = []
        if len(nodes) > 0:
            to_visit.append(nodes.pop())

len(communities)

6

In [6]:
data = []

for n, nbrdict in social_graph.adjacency():
    data.append([n, len(nbrdict)])
    
df = pd.DataFrame(data=data, columns=['node', 'degree'])

df.describe()

,node,degree
count,3307.000000,3307.000000
mean,1653.000000,43.217115
std,954.792997,74.060990
min,0.000000,0.000000
25%,826.500000,4.000000
50%,1653.000000,15.000000
75%,2479.500000,47.000000
max,3306.000000,616.000000


### Spotify 
'danceability' : 0

'energy' : 1

'loudness' : 2 //dividir -60

'mode' : 3

'speechiness' : 4

'acousticness' : 5

'instrumentalness' : 6

'liveness' : 7

'valence' : 8

'tempo' : 9 // Dividir 144

'key' : Extra

Ignora:
'duration_ms' : 258787,

'time_signature' : 4, 4 el 99% de las veces

'spotify_id' :

In [7]:
'''    
def spotify_feature_to_np(array, track_id, db_doc):
    array[track_id, 0] = db_doc['danceability']
    array[track_id, 1] = db_doc['energy']
    array[track_id, 2] = db_doc['loudness'] 
    array[track_id, 3] = db_doc['mode']
    array[track_id, 4] = db_doc['speechiness']
    array[track_id, 5] = db_doc['acousticness']
    array[track_id, 6] = db_doc['instrumentalness']
    array[track_id, 7] = db_doc['liveness']
    array[track_id, 8] = db_doc['valence']
    array[track_id, 9] = db_doc['tempo']
    pass


def compute_distance(graph, users, cos):
    sim = []
    for u in range(users):
        for t in graph.neighbors(u):
            sim.append(cos[u, t - users])
    sim = np.asarray(sim)
    mean = np.mean(sim)
    std = np.std(sim)
    return np.clip(1 - (cos - (mean - 2 * std)) / (4 * std), 0.1, 0.9)
    
    
class DataGenerator:
    
    def __init__(self, db, users_tracks, users, tracks, social, cos):
        self.users = users
        self.tracks = tracks 
        self.social = social
        self.distance = compute_distance(users_tracks, len(users), cos)
        self.users_count = len(users)
        self.track_count = len(users_tracks.nodes()) - len(users)
        self.process_tags_fast(db)
        self.process_spotify_fast(db)
        self.process_hat_d()
        pass   
    
    def map_id_tracks(self, artist_tracks):
        id_track = {}
        for a, tracks in artist_tracks.items():
            for t, idx in tracks.items():
                id_track[idx - self.users_count] = (a, t)
        l = [None] * len(id_track)
        for i, v in id_track.items():
            l[i] = v
        return l
    
    def process_tags_fast(self, db):
        print('Processing artists and tags...')
        id_track = self.map_id_tracks(self.tracks)
        track_id = {t: e for e, t in enumerate(id_track)}
        #Set of tags
        self.tag_id = {}
        self.tracks_tags = [None] * len(id_track)
        #Set of artist
        self.artist_id = {}
        self.tracks_artist = np.zeros((len(id_track), 1), dtype=np.int32) 
        #Load tag - tracks
        save = True
        if os.path.exists('data/tags_artist.pickle'):
            with open('data/tags_artist.pickle', 'rb') as f:
                data = pickle.load(f)
            self.tag_id = data['tag_id']
            self.artist_id = data['artist_id']
            save = False
        for r in tqdm(music.track_info.find({'spotify_id': {'$exists': True}}, 
                                            {'artist': True, 'track': True, 'tags': True}), 
                      total=music.track_info.count_documents({'spotify_id': {'$exists': True}})):
            if (r['artist'], r['track']) not in track_id:
                continue
            #Process tags
            tags = r['tags']
            t_ids = []
            for t in tags:
                if t not in self.tag_id:
                    if not save:
                        print('Tag not found: {}'.format(t))
                        continue
                    self.tag_id[t] = len(self.tag_id) + 1
                t_ids.append(self.tag_id[t])
            self.tracks_tags[track_id[(r['artist'], r['track'])]] = t_ids
            #Process Artist
            artist = r['artist']
            if artist not in self.artist_id:
                if not save:
                    print('Artist not found: {}'.format(artist))
                    continue
                self.artist_id[artist] = len(self.artist_id)
            self.tracks_artist[track_id[(r['artist'], r['track'])]] = self.artist_id[artist]
        if save:
            with open('data/tags_artist.pickle', 'wb') as f:
                pickle.dump({'tag_id': self.tag_id, 'artist_id': self.artist_id}, f)
        pass
    
    def process_spotify_fast(self, db):
        print('Processing spotify...')
        id_track = self.map_id_tracks(self.tracks)
        track_id = {t: e for e, t in enumerate(id_track)}
        #Load spotify to id track.
        spotify_track = {}
        for r in tqdm(music.track_info.find({'spotify_id': {'$exists': True}}, 
                                            {'artist': True, 'track': True, 'spotify_id': True}), 
                      total=music.track_info.count_documents({'spotify_id': {'$exists': True}})):
            if (r['artist'], r['track']) in track_id:
                spotify_track[r['spotify_id']] = (r['artist'], r['track'])
        #Load info into a numpy matrix
        self.track_spotify_features = np.zeros((len(track_id), 10))
        self.track_spotify_key = np.zeros((len(track_id), 1), dtype=np.int8)
        for spotify_features in tqdm(music.track_spotify_features.find({}),
                                     total=music.track_spotify_features.count_documents({})):
            if spotify_features['spotify_id'] not in spotify_track:
                continue
            t_id = track_id[spotify_track[spotify_features['spotify_id']]]
            spotify_feature_to_np(self.track_spotify_features, t_id, spotify_features)
            self.track_spotify_key[t_id, 0] = spotify_features['key'] 
        #Loudness de -60 a 4... Map clip(-60, -2e-4) log
        self.track_spotify_features[:, 2] = np.log10(-np.clip(self.track_spotify_features[:, 2], -60, -2e-4))
        #Tempo clipped -2, 2 -mean / stdb
        tempo_mean = np.mean(self.track_spotify_features[:, 9])
        tempo_stdev = np.std(self.track_spotify_features[:, 9])
        self.track_spotify_features[:, 9] = np.clip((self.track_spotify_features[:, 9] - tempo_mean) / tempo_stdev, -2, 2)
        pass
    
    def process_hat_d(self):
        print('Processing hat d')
        a = np.eye(len(self.social.nodes))
        for u in tqdm(self.social.nodes):
            for n in self.social.neighbors(u):
                a[u, n] = 1
        self.d_hat = normalized_adjacency(a)
        pass
    
    def get_users_data_slow(self, ids):
        users = 0
        for u in ids:
            neighbors = len(list(self.social.neighbors(u))) 
            if users < neighbors:
                users = neighbors
        users_ids = np.zeros((ids.shape[0], users + 1), dtype=np.int32)
        users_graph = np.zeros((ids.shape[0], 1, users + 1))
        for i, u in enumerate(ids):
            nodes = list(self.social.neighbors(u))
            neighbors = np.asarray(nodes)
            users_ids[i, 0] = u
            users_ids[i, 1:neighbors.shape[0] + 1] = neighbors
            #Sub Adjacency matrix for the first level neighbors
            #this is done because we need the normalize adyacency
            id_map = {u: e for e, u in enumerate(nodes, start=1)}
            id_map[u] = 0
            for u in list(id_map.keys()):
                for n in self.social.neighbors(u):
                    if n not in id_map:
                        id_map[n] = len(id_map)
            a = np.eye(len(id_map))
            for ui in [u] + list(self.social.neighbors(u)):
                for n in self.social.neighbors(ui):
                    a[id_map[ui], id_map[n]] = 1
            d = normalized_adjacency(a)
            users_graph[i, 0, :min(d.shape[1], users + 1)] = d[0, :min(d.shape[1], users + 1)]
        return [users_ids, users_graph]
    
    def get_users_data(self, ids):
        users = 0
        for u in ids:
            neighbors = len(list(self.social.neighbors(u))) 
            if users < neighbors:
                users = neighbors
        users_ids = np.zeros((ids.shape[0], users + 1), dtype=np.int32)
        users_graph = np.zeros((ids.shape[0], 1, users + 1))
        for i, u in enumerate(ids):
            nodes = list(self.social.neighbors(u))
            neighbors = np.asarray(nodes)
            users_ids[i, 0] = u
            users_ids[i, 1:neighbors.shape[0] + 1] = neighbors
            users_graph[i, 0, 0] = self.d_hat[u, u]
            users_graph[i, 0, 1:len(nodes) + 1] = self.d_hat[u, nodes]
        return [users_ids, users_graph]
        
    def get_tracks_data(self, ids):
        tracks_ids = ids[:, np.newaxis]
        #Tags
        tags = []
        for i in ids:
            tags.append(self.tracks_tags[i])
        tags_np = np.zeros((ids.shape[0], max([len(t) for t in tags])), dtype=np.int32)
        for i, t in enumerate(tags):
            tags_np[i, :len(t)] = t
        #Artist
        artists = self.tracks_artist[ids, :]
        return [tracks_ids, tags_np, artists]
'''

"    \ndef spotify_feature_to_np(array, track_id, db_doc):\n    array[track_id, 0] = db_doc['danceability']\n    array[track_id, 1] = db_doc['energy']\n    array[track_id, 2] = db_doc['loudness'] \n    array[track_id, 3] = db_doc['mode']\n    array[track_id, 4] = db_doc['speechiness']\n    array[track_id, 5] = db_doc['acousticness']\n    array[track_id, 6] = db_doc['instrumentalness']\n    array[track_id, 7] = db_doc['liveness']\n    array[track_id, 8] = db_doc['valence']\n    array[track_id, 9] = db_doc['tempo']\n    pass\n\n\ndef compute_distance(graph, users, cos):\n    sim = []\n    for u in range(users):\n        for t in graph.neighbors(u):\n            sim.append(cos[u, t - users])\n    sim = np.asarray(sim)\n    mean = np.mean(sim)\n    std = np.std(sim)\n    return np.clip(1 - (cos - (mean - 2 * std)) / (4 * std), 0.1, 0.9)\n    \n    \nclass DataGenerator:\n    \n    def __init__(self, db, users_tracks, users, tracks, social, cos):\n        self.users = users\n        self.tr

In [8]:
def spotify_feature_to_np(array, track_id, db_doc):
    array[track_id, 0] = db_doc['danceability']
    array[track_id, 1] = db_doc['energy']
    array[track_id, 2] = db_doc['loudness'] 
    array[track_id, 3] = db_doc['mode']
    array[track_id, 4] = db_doc['speechiness']
    array[track_id, 5] = db_doc['acousticness']
    array[track_id, 6] = db_doc['instrumentalness']
    array[track_id, 7] = db_doc['liveness']
    array[track_id, 8] = db_doc['valence']
    array[track_id, 9] = db_doc['tempo']
    pass


def compute_distance(graph, users, cos):
    sim = []
    for u in range(users):
        for t in graph.neighbors(u):
            sim.append(cos[u, t - users])
    sim = np.asarray(sim)
    mean = np.mean(sim)
    std = np.std(sim)
    return np.clip(1 - (cos - (mean - 2 * std)) / (4 * std), 0.1, 0.9)
    

def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str.lower())
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])


class DataGenerator:
    
    def __init__(self, db, users_tracks, users, tracks, social, cos):
        self.users = users
        self.tracks = tracks 
        self.social = social
        self.distance = compute_distance(users_tracks, len(users), cos)
        self.users_count = len(users)
        self.track_count = len(users_tracks.nodes()) - len(users)
        self.process_tags_fast(db)
        self.process_spotify_fast(db)
        self.process_hat_d()
        pass   
    
    def map_id_tracks(self, artist_tracks):
        id_track = defaultdict(list)
        for a, tracks in artist_tracks.items():
            for t, idx in tracks.items():
                id_track[idx - self.users_count].append((a, t))
        l = [None] * len(id_track)
        for i, v in id_track.items():
            l[i] = v
        return l
    
    def process_tags_fast(self, db):
        print('Processing artists and tags...')
        id_track = self.map_id_tracks(self.tracks)
        track_id = {}
        for e, tracks in enumerate(id_track):
            for t in tracks:
                track_id[t] = e
        #Set of tags
        tracks_tags = [set() for _ in range(len(id_track))] 
        #Set of artist
        tracks_artist = [set() for _ in range(len(id_track))] 
        #Load tag - tracks
        for r in tqdm(music.track_info.find({'spotify_id': {'$exists': True}}, 
                                            {'artist': True, 'track': True, 'tags': True}), 
                      total=music.track_info.count_documents({'spotify_id': {'$exists': True}})):
            if (r['artist'], r['track']) not in track_id:
                continue
            #Process tags
            tracks_tags[track_id[(r['artist'], r['track'])]].update([remove_accents(t) for t in r['tags']])
            #Process Artist
            tracks_artist[track_id[(r['artist'], r['track'])]].add(remove_accents(r['artist']))
        #Keep shortest artist
        for i in range(len(tracks_artist)):
            artists = list(tracks_artist[i])
            artists.sort(key=lambda x: (len(x), x))
            tracks_artist[i] = artists[0]
        #To ids
        if os.path.exists('data/tags_artist.pickle'):
            with open('data/tags_artist.pickle', 'rb') as f:
                data = pickle.load(f)
            self.tag_id = data['tag_id']
            self.artist_id = data['artist_id']
        else:
            self.artist_id = {a: e for e, a in enumerate(set(tracks_artist))}
            tags_counter = Counter()
            for tags in tracks_tags:
                for tag in tags:
                    tags_counter[tag] += 1
            self.tag_id = {}
            for tag, c in tags_counter.items():
                if c > 4:
                    self.tag_id[tag] = len(self.tag_id) + 1
            with open('data/tags_artist.pickle', 'wb') as f:
                pickle.dump({'tag_id': self.tag_id, 'artist_id': self.artist_id}, f)
        self.tracks_tags = [None] * len(tracks_tags)
        for i, tags in enumerate(tracks_tags):
            ids = [self.tag_id[t] for t in tags if t in self.tag_id]
            self.tracks_tags[i] = ids
        self.tracks_artist = np.zeros((len(id_track), 1), dtype=np.int32) 
        for e, a in enumerate(tracks_artist):
            self.tracks_artist[e, 0] = self.artist_id[a]
        pass
    
    def process_spotify_fast(self, db):
        print('Processing spotify...')
        id_track = self.map_id_tracks(self.tracks)
        track_id = {}
        for e, tracks in enumerate(id_track):
            for t in tracks:
                track_id[t] = e
        #Load spotify to id track.
        spotify_track = {}
        for r in tqdm(music.track_info.find({'spotify_id': {'$exists': True}}, 
                                            {'artist': True, 'track': True, 'spotify_id': True}), 
                      total=music.track_info.count_documents({'spotify_id': {'$exists': True}})):
            if (r['artist'], r['track']) in track_id and r['spotify_id'] not in spotify_track:
                spotify_track[r['spotify_id']] = track_id[(r['artist'], r['track'])]
        #Load info into a numpy matrix
        self.track_spotify_features = np.zeros((len(track_id), 10))
        self.track_spotify_key = np.zeros((len(track_id), 1), dtype=np.int8)
        for spotify_features in tqdm(music.track_spotify_features.find({}),
                                     total=music.track_spotify_features.count_documents({})):
            if spotify_features['spotify_id'] not in spotify_track:
                continue
            t_id = spotify_track[spotify_features['spotify_id']]
            spotify_feature_to_np(self.track_spotify_features, t_id, spotify_features)
            self.track_spotify_key[t_id, 0] = spotify_features['key'] 
        #Loudness de -60 a 4... Map clip(-60, -2e-4) log
        self.track_spotify_features[:, 2] = np.log10(-np.clip(self.track_spotify_features[:, 2], -60, -2e-4))
        #Tempo clipped -2, 2 -mean / stdb
        tempo_mean = np.mean(self.track_spotify_features[:, 9])
        tempo_stdev = np.std(self.track_spotify_features[:, 9])
        self.track_spotify_features[:, 9] = np.clip((self.track_spotify_features[:, 9] - tempo_mean) / tempo_stdev, -2, 2)
        pass
    
    def process_hat_d(self):
        print('Processing hat d')
        a = np.eye(len(self.social.nodes))
        for u in tqdm(self.social.nodes):
            for n in self.social.neighbors(u):
                a[u, n] = 1
        self.d_hat = normalized_adjacency(a)
        pass
    
    def get_users_data_slow(self, ids):
        users = 0
        for u in ids:
            neighbors = len(list(self.social.neighbors(u))) 
            if users < neighbors:
                users = neighbors
        users_ids = np.zeros((ids.shape[0], users + 1), dtype=np.int32)
        users_graph = np.zeros((ids.shape[0], 1, users + 1))
        for i, u in enumerate(ids):
            nodes = list(self.social.neighbors(u))
            neighbors = np.asarray(nodes)
            users_ids[i, 0] = u
            users_ids[i, 1:neighbors.shape[0] + 1] = neighbors
            #Sub Adjacency matrix for the first level neighbors
            #this is done because we need the normalize adyacency
            id_map = {u: e for e, u in enumerate(nodes, start=1)}
            id_map[u] = 0
            for u in list(id_map.keys()):
                for n in self.social.neighbors(u):
                    if n not in id_map:
                        id_map[n] = len(id_map)
            a = np.eye(len(id_map))
            for ui in [u] + list(self.social.neighbors(u)):
                for n in self.social.neighbors(ui):
                    a[id_map[ui], id_map[n]] = 1
            d = normalized_adjacency(a)
            users_graph[i, 0, :min(d.shape[1], users + 1)] = d[0, :min(d.shape[1], users + 1)]
        return [users_ids, users_graph]
    
    def get_users_data(self, ids):
        users = 0
        for u in ids:
            neighbors = len(list(self.social.neighbors(u))) 
            if users < neighbors:
                users = neighbors
        users_ids = np.zeros((ids.shape[0], users + 1), dtype=np.int32)
        users_graph = np.zeros((ids.shape[0], 1, users + 1))
        for i, u in enumerate(ids):
            #Drop of 25% chance
            nodes = list(self.social.neighbors(u))
            #print(nodes)
            neighbors = np.asarray(nodes)
            users_ids[i, 0] = u
            users_ids[i, 1:neighbors.shape[0] + 1] = neighbors
            users_graph[i, 0, 0] = self.d_hat[u, u]
            #Old users_graph[i, 0, 1:len(nodes) + 1] = self.d_hat[u, nodes]
            users_graph[i, 0, 1:len(nodes) + 1] = np.where(np.random.rand(len(nodes))  < 0.9, self.d_hat[u, nodes], 0)
            #print('******** users_graph')
            #print(users_graph[i,0,:])
            #print('******** d_hat')
            #print(self.d_hat[u, nodes])
            #print(f'{u} {nodes}')
            #print('********')
        return [users_ids, users_graph]
        
    def get_tracks_data(self, ids):
        tracks_ids = ids[:, np.newaxis]
        #Tags
        tags = []
        for i in ids:
            tags.append(self.tracks_tags[i])
        tags_np = np.zeros((ids.shape[0], max([len(t) for t in tags])), dtype=np.int32)
        for i, t in enumerate(tags):
            tags_np[i, :len(t)] = t
        #Artist
        artists = self.tracks_artist[ids, :]
        spotify = self.track_spotify_features[ids, ...]
        return [tracks_ids, tags_np, artists, spotify]
    

In [9]:
def search_superior(data, percentage=0.01, dmin=None, dmax=None):
    total = data.shape[0]
    if dmin is None:
        dmin = np.min(data)
    if dmax is None:
        dmax = np.max(data)
    mid = (dmax + dmin) // 2
    while dmin != mid and mid != dmax:
        p = np.sum(data >= mid) / total
        print(f'{p} {dmin} {mid} {dmax}')
        if p > percentage:
            dmin = mid
        elif p < percentage:
            dmax = mid
        else:
            return mid
        mid = (dmax + dmin) // 2
    pmin = np.sum(data > dmin) / total
    pmax = np.sum(data > dmax) / total
    if abs(pmin - percentage) < abs(pmax - percentage):
        return dmin
    return dmax

class TrainGenerator(DataGenerator, Sequence):
    
    def __init__(self, db, users_tracks, users, tracks, social, cos, batch_size=32, neg_sampling=1, discard_top=None):
        super().__init__(db, users_tracks, users, tracks, social, cos)
        self.discard_top = discard_top
        self.users_tracks = users_tracks
        self.batch_size = batch_size
        self.neg_sampling = neg_sampling + 1
        self.process_user_song()
        self.shuffle_indexes = list(range(self.iduser_idtrack.shape[0]))
        self.on_epoch_end()
   
    def process_user_song(self):
        print('Processing users-song...')
        data_size = 0
        for u in range(len(self.users)):
            data_size += len(list(self.users_tracks.neighbors(u)))
        idx = 0
        user_track = np.zeros((data_size, 2), dtype=np.int32) 
        data = np.zeros((data_size, 2))
        with tqdm(total=data_size) as pbar:
            for u in range(len(self.users)):
                for t, atts in self.users_tracks[u].items():
                    t_id = t - len(self.users)
                    user_track[idx, 0] = u
                    user_track[idx, 1] = t_id
                    data[idx, 0] = atts['scrobbles']
                    data[idx, 1] = self.distance[u, t_id]
                    idx += 1
                    pbar.update(1)
        self.iduser_idtrack = user_track
        if self.discard_top is not None:
            limit = search_superior(data[:, 0], percentage=self.discard_top)
            data[:, 0] = np.clip(data[:, 0], 0, limit)
        self.scrobble_cos = data
        pass
    
    def on_epoch_end(self):
        random.shuffle(self.shuffle_indexes)
    
    def __len__(self):
        return math.ceil(self.neg_sampling * len(self.shuffle_indexes) / self.batch_size)
    
    def get_neg_sampling(self):
        users = np.random.randint(0, self.users_count, size=(self.batch_size,))
        tracks = np.random.randint(0, self.track_count, size=(self.batch_size,))
        users_data = self.get_users_data(users)
        tracks_data = self.get_tracks_data(tracks)
        y = np.empty((self.batch_size, 2))
        y[:, 0] = 0.5
        y[:, 1] = 1 - self.distance[users, tracks]
        return users_data + tracks_data, y
    
    def __getitem__(self, index):
        if index % self.neg_sampling == 0:
            return self.get_neg_sampling()
        index = index // self.neg_sampling
        indexes = self.shuffle_indexes[index * self.batch_size: (index + 1) * self.batch_size]
        users_tracks = self.iduser_idtrack[indexes, ...]
        users_data = self.get_users_data(users_tracks[:, 0])
        tracks_data = self.get_tracks_data(users_tracks[:, 1])
        y = self.scrobble_cos[indexes, ...]
        return users_data + tracks_data, y

In [10]:
cosines = np.load('data/cos.npz')['cosines']

In [11]:
ds = TrainGenerator(music, dataset['train'], dataset['users'], 
                   dataset['artist-tracks'], social_graph,
                    cosines, discard_top=0.03, batch_size=512)

Processing artists and tags...


  0%|          | 0/361499 [00:00<?, ?it/s]

Processing spotify...


  0%|          | 0/361499 [00:00<?, ?it/s]

  0%|          | 0/348947 [00:00<?, ?it/s]

Processing hat d


  0%|          | 0/3307 [00:00<?, ?it/s]

Processing users-song...


  0%|          | 0/2564908 [00:00<?, ?it/s]

2.3392651900185114e-06 1.0 102845.0 205689.0
5.0684079117067745e-06 1.0 51423.0 102845.0
8.18742816506479e-06 1.0 25712.0 51423.0
1.871412152014809e-05 1.0 12856.0 25712.0
3.898775316697519e-05 1.0 6428.0 12856.0
0.00012593044272932985 1.0 3214.0 6428.0
0.0006167862551015476 1.0 1607.0 3214.0
0.0024164609412891222 1.0 804.0 1607.0
0.008821758909091476 1.0 402.0 804.0
0.03042721220410245 1.0 201.0 402.0
0.014852774446490868 201.0 301.0 402.0
0.020697818401283787 201.0 251.0 301.0
0.024889001866733623 201.0 226.0 251.0
0.027585004998229957 201.0 213.0 226.0
0.02896322207268253 201.0 207.0 213.0
0.029709057790766765 201.0 204.0 207.0
0.030190166664847237 201.0 202.0 204.0
0.029947272962616983 202.0 203.0 204.0


In [12]:
for i in ds[0][0]:
    print(i.shape)

(512, 617)
(512, 1, 617)
(512, 1)
(512, 16)
(512, 1)
(512, 10)


In [13]:
ds.d_hat[464, [943, 856, 538, 485, 1015, 634, 948, 0]]

array([0.35355339, 0.20412415, 0.14433757, 0.08838835, 0.20412415,
       0.1767767 , 0.25      , 0.        ])

# Red

In [14]:
def loss(y_true, y_pred):
    #recibe indices con forma 1xvaloresx3 (indices + valor)
    #trasnforma los indices a valoresx2 y los valores valoresx1
    v_true, dist = y_true[:, 0], y_true[:, 1]
    dist = dist ** 0.75
    v_true = K.expand_dims(v_true, axis=-1)
    dist = K.expand_dims(dist, axis=-1)
    return K.mean(dist * K.square(y_pred - K.log(2 * v_true) / K.log(2.0)))

In [15]:
emb_size = 64
kernel_size = 32
deep = 1

#Input users
i_users = Input((None,), name='users')
i_user_graph = Input((None, None), name='user_graph')

#Input music
i_tracks = Input((1,), name='tracks')
i_tags = Input((None,), name='tags')
i_artist = Input((1,), name='artist')
i_spotify = Input((10,), name='spotify')

#Process users
emb_user_base = Embedding(len(dataset['users']), emb_size, name="embedding_users")(i_users)
emb_user = emb_user_base
emb_user_base = Lambda(lambda x: x[:, 0,:], name='extract_emb_user_base')(emb_user_base)
for i in range(deep):
    emb_user = GCNConv(kernel_size, name='gcn_user_{}'.format(i))([emb_user, i_user_graph])

emb_user = Lambda(lambda x: x[:, 0,:], name='extract_gcn_user')(emb_user)
#Process music
def avg(x):
    i = x[0]
    m = x[1]
    i = i * tf.expand_dims(tf.cast(m, tf.float32), axis=-1)
    r = tf.reduce_sum(i, axis=-2) / tf.expand_dims(tf.reduce_sum(tf.cast(m, tf.float32), axis=-1), axis=-1)
    return tf.where(tf.math.logical_or(tf.math.is_nan(r), tf.math.is_inf(r)), 0., r)

emb_tracks = Embedding(len(dataset['train'].nodes()) - len(dataset['users']), emb_size, name="embedding_tracks")(i_tracks)
emb_tracks = Lambda(lambda x: x[:, 0, :], name='extract_track')(emb_tracks)

mask_tags = Lambda(lambda x: x != 0, name='mask_tags')(i_tags)
emb_tags = Embedding(len(ds.tag_id) + 1, emb_size, name='embedding_tags')(i_tags)
emb_tags = Lambda(avg, name='masked_average_tags')([emb_tags, mask_tags])

emb_artist = Embedding(len(ds.artist_id), emb_size, name='embedding_artist')(i_artist)
emb_artist = Lambda(lambda x: x[:, 0, :], name='extract_artist')(emb_artist)

emb_music = Concatenate(name='concatenate_embedding_track_tag_artist')([emb_tracks, emb_tags, emb_artist, i_spotify])
#emb_music = Dense(kernel_size, name='dense_music')(emb_music)

#Deep part
deep = Concatenate(name='concatenate_gcn_user_music')([emb_user_base, emb_user, emb_music])
deep = Dense(256, name='deep_dense_1')(deep)
deep = Dense(256, name='deep_dense_2')(deep)
deep = Dense(1, name='deep_dense_3')(deep)

#Wide 
wide = Concatenate(name='concatentate_user_track')([emb_user_base, emb_tracks, emb_tags, emb_artist])
wide = Dense(1, name='wide')(wide)

out = Add(name='deep_plus_wide')([deep, wide])

model = Model([i_users, i_user_graph, i_tracks, i_tags, i_artist, i_spotify], out)

model.summary()

model.compile(loss=loss, optimizer='adam')

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
users (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
tracks (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
tags (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
artist (InputLayer)             [(None, 1)]          0                                            
_______________________________________________________________________________________

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import os
if not os.path.exists('model_dropout_complex_075'):
    os.makedirs('model_dropout_complex_075')

def scheduler(epoch, lr):
    if epoch <= 5 or epoch % 2 != 0:
        return lr
    return lr * tf.math.exp(-0.1)


model.fit(ds, epochs=10, callbacks=[ModelCheckpoint(filepath='model_dropout_complex_075/{epoch:02d}-model-{loss:.5f}.hdf5',
                                                        monitor='loss',
                                                        save_best_only=False),
                                         LearningRateScheduler(scheduler)], workers=1)

Epoch 1/10
  464/10020 [>.............................] - ETA: 1:07:18 - loss: 2.0441

In [ ]:
model.save_weights('model_dropout_complex_075/model.h5')